In [ ]:
import pandas as pd
import numpy as np
 
import seaborn as sns
from matplotlib import pyplot as plt

Get The Data: New York City Airbnb Open Data

In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv

--2021-09-16 09:41:11--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘AB_NYC_2019.csv.1’

AB_NYC_2019.csv.1   100%[===================>]   6.75M  --.-KB/s    in 0.05s   

2021-09-16 09:41:12 (140 MB/s) - ‘AB_NYC_2019.csv.1’ saved [7077973/7077973]



In [ ]:
df = pd.read_csv('AB_NYC_2019.csv')
len(df)

48895

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
used_columns = [
                'latitude',
                'longitude',
                'price',
                'minimum_nights',
                'number_of_reviews',
                'reviews_per_month',
                'calculated_host_listings_count',
                'availability_365'   
                ]

In [ ]:
HW_df = df[used_columns]
HW_df.head(3)

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64749,-73.97237,149,1,9,0.21,6,365
1,40.75362,-73.98377,225,1,45,0.38,2,355
2,40.80902,-73.94190,150,3,0,NaN,1,365


------------------------------------------------------------------------------
ANS Q1:

In [ ]:
HW_df.isnull().sum()

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

------------------------------------------------------------------------------
ANS Q2:

In [ ]:
HW_df['minimum_nights'].median()

3.0

------------------------------------------------------------------------------
Split Data (Manually)

In [ ]:
def split_data(n,split_val,split_test):
  n_val = int(split_val*n)
  n_test = int(split_test*n)
  n_train = n - n_val - n_test
  print('Train Split : ', 1 - split_val - split_test)
  print(' ')
  print(n_train, n_val, n_test)
  
  return n_train, n_val, n_test

In [ ]:
n = len(HW_df)

split_val = float(input('Val Split : '))
split_test = float(input('Test Split : '))

n_train, n_val, n_test = split_data(n,split_val,split_test)

Val Split : 0.2
Test Split : 0.2
Train Split :  0.6000000000000001
 
29337 9779 9779


In [ ]:
idx = np.arange(n)

np.random.seed(42)
np.random.shuffle(idx)

In [ ]:
df_train = HW_df.iloc[idx[:n_train]]
df_val = HW_df.iloc[idx[n_train:n_train+n_val]]
df_test = HW_df.iloc[idx[n_train+n_val:]]

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
df_train.head(3)

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64354,-73.97777,89,3,62,0.71,1,189
1,40.70666,-73.90779,30,21,0,NaN,1,73
2,40.76116,-73.99016,120,2,17,0.43,1,0


In [ ]:
y_train = np.log1p(df_train.price.values)
y_val = np.log1p(df_val.price.values)
y_test = np.log1p(df_test.price.values)

In [ ]:
y_train

array([4.49980967, 3.4339872 , 4.79579055, ..., 4.61512052, 4.2341065 ,
       3.8286414 ])

In [ ]:
del df_train['price']
del df_val['price']
del df_test['price']

In [ ]:
len(y_train)

29337

In [ ]:
df_train.head(3)

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,40.64354,-73.97777,3,62,0.71,1,189
1,40.70666,-73.90779,21,0,NaN,1,73
2,40.76116,-73.99016,2,17,0.43,1,0


------------------------------------------------------------------------------
Linear Regression (w/o Regularization)

In [ ]:
def LG_wo_R(X, y):
  ones = np.ones(X.shape[0])
  X = np.column_stack([ones, X])

  XTX = X.T.dot(X)
  XTX_inv = np.linalg.inv(XTX)
  w = XTX_inv.dot(X.T).dot(y)

  return w[0], w[1:]

------------------------------------------------------------------------------
RMSE

In [ ]:
def RMSE(y, y_pred):
  e = y_pred - y
  mse = (e ** 2).mean()
  rmse_val = np.sqrt(mse)

  return rmse_val

------------------------------------------------------------------------------
ANS Q3:
For this question there are 2 options


1.   Fill the Missing Value with 0
2.   Fill the Missing Value with mean



In [ ]:
base = [
        'latitude',
        'longitude',
        'minimum_nights',
        'number_of_reviews',
        'reviews_per_month',
        'calculated_host_listings_count',
        'availability_365'
        ]

*Option 1*

In [ ]:
def X_opt_1(df):
  df_opt_1 = df[base]
  df_opt_1 = df_opt_1.fillna(0)
  X = df_opt_1.values

  return X

In [ ]:
X1_train = X_opt_1(df_train)
w0_1, w_1 = LG_wo_R(X1_train, y_train)

In [ ]:
y1_pred = w0_1 + X1_train.dot(w_1)

In [ ]:
print('RMSE value = ', round(RMSE(y_train, y1_pred), 2))

RMSE value =  0.64


*Option 2*

In [ ]:
def X_opt_2(df):
  df_opt_1 = df[base]
  df_opt_1 = df_opt_1.fillna(float(HW_df['reviews_per_month'].mean()))
  X = df_opt_1.values

  return X

In [ ]:
X2_train = X_opt_2(df_train)
w0_2, w_2 = LG_wo_R(X2_train, y_train)

In [ ]:
y2_pred = w0_2 + X2_train.dot(w_2)

In [ ]:
print('RMSE value = ', round(RMSE(y_train, y2_pred), 2))

RMSE value =  0.64


-------------------------------------------------------------------------------
Regulized Linear Regression

In [ ]:
def LG_w_R(X, y, r=0.0):
  ones = np.ones(X.shape[0])
  X = np.column_stack([ones, X])

  XTX = X.T.dot(X)
  reg = r * np.eye(XTX.shape[0])
  XTX += reg
  
  XTX_inv = np.linalg.inv(XTX)
  w = XTX_inv.dot(X.T).dot(y)

  return w[0], w[1:]

---------------------------------------------------------------------------------------------
ANS Q4:

In [ ]:
X_train_q4 = X_opt_1(df_train)
X_val_q4 = X_opt_1(df_val)

In [ ]:
for r_q4 in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
  w0_q4, w_q4 = LG_w_R(X_train_q4, y_train, r=r_q4)
  y_pred_q4 = w0_q4 + X_val_q4.dot(w_q4)
  print('r = %7s' %r_q4, '-> RMSE value = ', round(RMSE(y_val, y_pred_q4), 2))

r =       0 -> RMSE value =  0.64
r =   1e-06 -> RMSE value =  0.64
r =  0.0001 -> RMSE value =  0.64
r =   0.001 -> RMSE value =  0.64
r =    0.01 -> RMSE value =  0.66
r =     0.1 -> RMSE value =  0.68
r =       1 -> RMSE value =  0.68
r =       5 -> RMSE value =  0.68
r =      10 -> RMSE value =  0.68


------------------------------------------------------------------------------
ANS Q5:

In [ ]:
n_q5 = len(HW_df)

split_val_q5 = float(input('Val Split : '))
split_test_q5 = float(input('Test Split : '))

n_train_q5, n_val_q5, n_test_q5 = split_data(n_q5,split_val_q5,split_test_q5)

Val Split : 0.2
Test Split : 0.2
Train Split :  0.6000000000000001
 
29337 9779 9779


In [ ]:
for s in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
  idx_q5 = np.arange(n)

  np.random.seed(s)
  np.random.shuffle(idx_q5)

  df_train_q5 = HW_df.iloc[idx_q5[:n_train_q5]]
  df_val_q5 = HW_df.iloc[idx_q5[n_train_q5:n_train_q5+n_val_q5]]
  df_test_q5 = HW_df.iloc[idx_q5[n_train_q5+n_val_q5:]]

  df_train_q5 = df_train_q5.reset_index(drop=True)
  df_val_q5 = df_val_q5.reset_index(drop=True)
  df_test_q5 = df_test_q5.reset_index(drop=True)

  y_train_q5 = np.log1p(df_train_q5.price.values)
  y_val_q5 = np.log1p(df_val_q5.price.values)
  y_test_q5 = np.log1p(df_test_q5.price.values)  

  del df_train_q5['price']
  del df_val_q5['price']
  del df_test_q5['price']

  X_train_q5 = X_opt_1(df_train_q5)
  X_val_q5 = X_opt_1(df_val_q5)
  
  w0_q5, w_q5 = LG_wo_R(X_train_q5, y_train_q5)
  y_pred_q5 = w0_q5 + X_val_q5.dot(w_q5)
  RMSE_score = round(RMSE(y_val, y_pred_q5), 4)

  print('s = %1s' %s, '-> RMSE value = ', RMSE_score)

s = 0 -> RMSE value =  0.7427
s = 1 -> RMSE value =  0.7387
s = 2 -> RMSE value =  0.7427
s = 3 -> RMSE value =  0.7381
s = 4 -> RMSE value =  0.7403
s = 5 -> RMSE value =  0.7434
s = 6 -> RMSE value =  0.7393
s = 7 -> RMSE value =  0.7415
s = 8 -> RMSE value =  0.7418
s = 9 -> RMSE value =  0.7434


In [ ]:
scores = [0.7427, 0.7387, 0.7427, 0.7381, 0.7403, 0.7434, 0.7393, 0.7415, 0.7418, 0.7434]
scores = np.array(scores)

In [ ]:
print('Standard  Deviation = ', round(np.std(scores), 3))

Standard  Deviation =  0.002


------------------------------------------------------------------------------
ANS Q6:

In [ ]:
n_q6 = len(HW_df)

split_val_q6 = float(input('Val Split : '))
split_test_q6 = float(input('Test Split : '))

n_train_q6, n_val_q6, n_test_q6 = split_data(n_q6,split_val_q6,split_test_q6)

Val Split : 0.2
Test Split : 0.2
Train Split :  0.6000000000000001
 
29337 9779 9779


In [ ]:
idx_q6 = np.arange(n)

np.random.seed(9)
np.random.shuffle(idx_q6)

df_train_q6 = HW_df.iloc[idx_q5[:n_train_q6+n_val_q6]]
df_test_q6 = HW_df.iloc[idx_q5[n_train_q6+n_val_q6:]]

df_train_q6 = df_train_q6.reset_index(drop=True)
df_test_q6 = df_test_q6.reset_index(drop=True)

y_train_q6 = np.log1p(df_train_q6.price.values)
y_test_q6 = np.log1p(df_test_q6.price.values) 

del df_train_q6['price']
del df_test_q6['price']

X_train_q6 = X_opt_1(df_train_q6)
X_test_q6 = X_opt_1(df_test_q6)
  
w0_q6, w_q6 = LG_w_R(X_train_q6, y_train_q6, r=0.001)
y_pred_q6 = w0_q6 + X_test_q6.dot(w_q6)
RMSE_score = round(RMSE(y_test_q6, y_pred_q6), 2)

print('RMSE value = ', RMSE_score)

RMSE value =  0.65
